For ent hashtags, I could do the clustering with OPTICS that do not need number of hashtags in advance. Then for finding the relevant clusters by OPTICS we can just find the clusters having the word "entrepreneur" in thier hashtags. For other clusters we cannot say without human intervention whether they are relevant or not since in contrast to kmeans, we do not have a center for the clusters so that we can find near clusters to target clusters.
As it is seen in the results, many hashtags are considered as noise (28058 out of 47088 hashtags). Although in some cluserts that have the keyword "entrepreneur" we can find some beliefs, there are many hashtags in those clusters that do seem irrelevant. Also, for other clusters we cannot give any opinion with human intervention. 

For the mng hashtags, Iused the kemans algorithm. With kmeans clustering with 100 clusters for mng hashtags it is seen that for clusters that have 'manager' or 'management', there are hashtags that seems irrelevant like kids, drones, .... If we want to change number of clusters to get the better results, there are two problems: 1) there is no guarantee  that it works better 2) we have human intervention. In other words, a human selects relevant hashtags not an algorithm.

So in general, clutering does not work for hashtags for finding beliefs.

In [1]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np
from sklearn.cluster import OPTICS
import time 
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import sys
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jafar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#accessing to tweets 
with open("tweet_ent.txt", "rb") as fp:   # Unpickling
  tweet_ent = pickle.load(fp)
  
with open("tweet_mng.txt", "rb") as fp:   # Unpickling
  tweet_mng = pickle.load(fp)

In [3]:
def cleaning (text):

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   
    
    #replace consecutive non-ASCII characters with a space
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)
    
    #removing hashtag sign
    text=re.sub('#', '', text) 
    
#    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}`;•«,@:~!=%&]+", ' ',text) # except ' ’ _
    
#    text=re.sub(' +',' ',text)
    
    text=''.join(text)
   
    return text

In [63]:
ent_hash_num=0
ent_hash_tweet=[]
for tweet in tweet_ent:
    ans=[]
    for  i in tweet.split():
        if i.startswith("#"):
            ans.extend(cleaning(i).split())
    if ans!=[]:
#        ans=' '.join(ans)
#        ent_hash_tweet.append(ans)
        ent_hash_tweet.extend(ans)
        ent_hash_num +=1
print("number of hashtags in ent:",len(ent_hash_tweet))         

number of hashtags in ent: 47088


In [64]:
mng_hash_num=0
mng_hash_tweet=[]
for tweet in tweet_mng:
    ans=[]
    for  i in tweet.split():
        if i.startswith("#"):
            ans.extend(cleaning(i).split())
    if ans!=[]:
#        ans=' '.join(ans)
#        mng_hash_tweet.append(ans)
        mng_hash_tweet.extend(ans)
        mng_hash_num +=1
print("number of hashtags in mng:",len(mng_hash_tweet)) 


number of hashtags in mng: 33771


In [6]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True) 

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [8]:
t0 = time.time()

ent_embedding=[]
ent_tokenized=[]

for text in ent_hash_tweet:
#    print(text)
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
#    print(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
#        print(type(outputs))
        out = outputs[0]
#        print(type(out))
    token_embeddings = torch.squeeze(out, dim=0)  
    
    token_embeddings=token_embeddings.numpy()
#    print(len(token_embeddings))
    
    tokenized_text.pop(0)
    tokenized_text.pop(-1)
    
    token_embeddings=np.delete(token_embeddings,0,axis=0)
    token_embeddings=np.delete(token_embeddings,-1,axis=0)
#    print(len(token_embeddings[0]))
    
    for ii in range(len(tokenized_text)-1 ,0 ,-1):
#        print(ii)
#        print(tokenized_text[ii][0:2])
        if tokenized_text[ii][0:2]=="##":
#            print(tokenized_text[ii])
            token_embeddings[ii-1]=token_embeddings[ii-1]+token_embeddings[ii]
            token_embeddings=np.delete(token_embeddings,ii,axis=0) 
            tokenized_text[ii-1]=tokenized_text[ii-1]+tokenized_text[ii][2:]
            tokenized_text.pop(ii) # or del tokenized_text[ii]
            
#    print("kll",len(token_embeddings[0]))
    ent_embedding.append(token_embeddings)
    ent_tokenized.append(tokenized_text)
#    print(ent_tokenized)
#    print()
    
    #for i, token_str in enumerate(tokenized_text):
    #    print (i, token_str)
    
print( time.time() - t0)

738.0343749523163


In [9]:
t0 = time.time()

mng_embedding=[]
mng_tokenized=[]

for text in mng_hash_tweet:
#    print(text)
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
#    print(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
#        print(type(outputs))
        out = outputs[0]
#        print(type(out))
    token_embeddings = torch.squeeze(out, dim=0)  
    
    token_embeddings=token_embeddings.numpy()
#    print(len(token_embeddings))
    
    tokenized_text.pop(0)
    tokenized_text.pop(-1)
    
    token_embeddings=np.delete(token_embeddings,0,axis=0)
    token_embeddings=np.delete(token_embeddings,-1,axis=0)
#    print(len(token_embeddings[0]))
    
    for ii in range(len(tokenized_text)-1 ,0 ,-1):
#        print(ii)
#        print(tokenized_text[ii][0:2])
        if tokenized_text[ii][0:2]=="##":
#            print(tokenized_text[ii])
            token_embeddings[ii-1]=token_embeddings[ii-1]+token_embeddings[ii]
            token_embeddings=np.delete(token_embeddings,ii,axis=0) 
            tokenized_text[ii-1]=tokenized_text[ii-1]+tokenized_text[ii][2:]
            tokenized_text.pop(ii) # or del tokenized_text[ii]
            
#    print("kll",len(token_embeddings[0]))
    mng_embedding.append(token_embeddings)
    mng_tokenized.append(tokenized_text)
#    print(mng_tokenized)
#    print()
    
    #for i, token_str in enumerate(tokenized_text):
    #    print (i, token_str)
    
print( time.time() - t0)

536.7798509597778


In [10]:
with open("hashtag_ent_embedding.txt", "wb") as fp:   #Pickling
  pickle.dump(ent_embedding, fp)

with open("hashtag_ent_tokenized.txt", "wb") as fp:   #Pickling
  pickle.dump(ent_tokenized, fp)

In [11]:
with open("hashtag_mng_embedding.txt", "wb") as fp:   #Pickling
  pickle.dump(mng_embedding, fp)

with open("hashtag_mng_tokenized.txt", "wb") as fp:   #Pickling
  pickle.dump(mng_tokenized, fp)

In [12]:
ent_embed_vec=[]
for vec in ent_embedding:
#    print(len(vec[0]))
    ent_embed_vec.append(vec[0])

In [13]:
np.shape(ent_embed_vec)

(47088, 768)

In [16]:
mng_embed_vec=[]
for vec in mng_embedding:
#    print(len(vec[0]))
    mng_embed_vec.append(vec[0]) 

In [17]:
np.shape(mng_embed_vec)

(33771, 768)

In [19]:
t0 = time.time()

ent_clustering = OPTICS(min_samples=10).fit(ent_embed_vec)
#ent_full_clustering = DBSCAN(eps=3, min_samples=2).fit(ent_embed_vec)
#kmeans = KMeans(n_clusters=100) 
#ent_full_clustering = kmeans.fit(ent_embed_vec)

print( time.time() - t0)

36394.50098347664


C:\Users\jafar\Anaconda3\lib\site-packages\sklearn\cluster\optics_.py:791: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [34]:
np.unique(ent_clustering.labels_)
print("number of hashtags in ent:",len(ent_hash_tweet))
ent_len_optics=len(np.unique(ent_clustering.labels_))
print("number of clusters in ent found by OPTICS clustering:",ent_len_optics-1,"+ 1 cluster for noise")

number of hashtags in ent: 47088
number of clusters in ent found by OPTICS clustering: 764 + 1 cluster for noise


By OPTICS we can just find the clusters having the word "entrepreneur" in thier hashtags. For other clusters we cannot say without human intervention whether they are relevant or not since in contrast to kmeans, we do not have a center for the clusters so that we can find near clusters to target clusters.

In the below we find just clusters that have the "entrepreneur" in thier hashatgs. We have shown all hashtags in these clusters except for cluster -1. Please note that clusters -1 means noise. As it is seen many hashtags are considered as noise (28058 out of 47088 hashtags).  

In [49]:
ent_optics_labels=np.unique(ent_clustering.labels_)

for label in ent_optics_labels:
#    print('cluster:',label)
    bb=np.where(ent_clustering.labels_==label)
#    print('number of members:',len(list(bb[0])))
    ind=list(bb[0])
#    print(ind)
    target=0
    for j in ind:
        if 'entrepreneur' in ent_hash_tweet[j]:
            
#            print(ent_hash_tweet[j])
            target +=1
    if target >=1:
        print('cluster:',label)
        print('number of members:',len(list(bb[0])))
        print("number of occurance of 'entrepreneur' in the clsuter:",target) 
        if label !=-1:
            for j in ind:
                print(ent_hash_tweet[j],end=', ')
        print()        
        print('****'*5)

cluster: -1
number of members: 28058
number of occurance of 'entrepreneur' in the clsuter: 60

********************
cluster: 9
number of members: 140
number of occurance of 'entrepreneur' in the clsuter: 109
Entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, Entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, entrepreneurs, Entrepreneur

In [52]:
t0 = time.time()

#mng_clustering = OPTICS(min_samples=10).fit(mng_embed_vec)
#ent_full_clustering = DBSCAN(eps=3, min_samples=2).fit(mng_embed_vec)
kmeans = KMeans(n_clusters=100) 
mng_clustering = kmeans.fit(mng_embed_vec)

print( time.time() - t0)

1344.2640557289124


In [68]:
print("number of hashtags in mng dataset:",len(mng_hash_tweet))
print()
mng_kmeans_labels=np.unique(mng_clustering.labels_)

for label in mng_kmeans_labels:
    print('cluster:',label)
    bb=np.where(mng_clustering.labels_==label)
    print('number of members:',len(list(bb[0])))
    ind=list(bb[0]) 
#    print(ind)
    target=0
    
    for j in ind:
        if ('manager' in mng_hash_tweet[j]) or ( 'management' in mng_hash_tweet[j]):
            
#            print(mng_hash_tweet[j])
            target +=1
    print("number of occurance of 'manager' or 'management' in the clsuter:",target) 
    if target >=1:
        for j in ind:
            print(ent_hash_tweet[j],end=', ')
        print()        
    print('****'*5)

number of hashtags in mng dataset: 33771

cluster: 0
number of members: 101
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 1
number of members: 1207
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 2
number of members: 170
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 3
number of members: 669
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 4
number of members: 737
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 5
number of members: 234
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 6
number of members: 71
number of occurance of 'manager' or 'management' in the clsuter: 0
********************
cluster: 7
number of members: 145
number of occurance of 'manager' or 'management' in the clsuter: 0
***

With kmeans clustering with 100 clusters for mng hashtags it is seen that for clusters that have 'manager' or 'management', there are hashtags that seems irrelevant like kids, drones, .... If we want to change number of clusters to get the better results, there are two problems: 1) there is no guarantee  that it works better 2) we have human intervention. In other words, a human selects relevant hashtags not an algorithm.